Evaluate the performance of different types of optimizer on a LeNet-5 network using MNIST data. At least you need to evaluate SGD, AdaGrad, RMSprop. 

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

In [2]:
import numpy as np
from datetime import datetime 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

import matplotlib.pyplot as plt

# check device
#DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
# Preparing for Data
print('==> Preparing data..')

"""
# Training Data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Testing Data preparation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

"""

==> Preparing data..


"\n# Training Data augmentation\ntransform_train = transforms.Compose([\n    transforms.RandomCrop(32, padding=4),\n    transforms.RandomHorizontalFlip(),\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n# Testing Data preparation\ntransform_test = transforms.Compose([\n    transforms.ToTensor(),\n    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),\n])\n\n#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')\n\n"

In [4]:
# Preparing for Data
print('==> Preparing data..')

# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])


==> Preparing data..


In [5]:
#Defining the convolutional neural network
class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        out = self.pool(F.relu(self.conv1(x)))
        out = self.pool(F.relu(self.conv2(out)))
        out = torch.flatten(out, 1) # flatten all dimensions except batch
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
 
        return out

In [6]:
model1 = LeNet()

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model1.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model1):,} trainable parameters')

The model has 61,706 trainable parameters


In [8]:
########################################################################
# 3. Define a Loss function and optimizer
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# Let's use a Classification Cross-Entropy loss and SGD with momentum.

#import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [9]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    count = 0
    train_loss = 0
    total = 0 
    correct = 0
    
    total_step = len(train_loader)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        

        #forward pass
        y_pred = model(data)

        loss = criterion(y_pred, target)

        acc = calculate_accuracy(y_pred, target)
        
        
        #Backward pass
        optimizer.zero_grad()
        
        loss.backward()

        optimizer.step()
        
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            

In [10]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [12]:
#checking the dataset in the code 
trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

In [13]:
#printing the dataset and dimensions to check whether we are working on the correct dataset:

for data, target in train_loader:
    print('Image batch dimensions:', data.shape)
    print('Image label dimensions:', target.shape)
    print('Class labels of the 10 examples:', target[:10])
    break
    

Image batch dimensions: torch.Size([128, 1, 32, 32])
Image label dimensions: torch.Size([128])
Class labels of the 10 examples: tensor([1, 4, 3, 5, 5, 2, 9, 7, 8, 1])


In [14]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 15
    lr = 0.01
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='mnist_data', train=False, download=True, transform=transforms)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"mnist_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298496
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.310501
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.301671
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.295326
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.285033
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.289911
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.288626
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.269981
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.262037
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.230506
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.167533
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.059894
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.701774
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.141136
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.786712
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.607631
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.385042
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.328181
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.277293
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.382361
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.031588
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.021705
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.042743
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.055931
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.114060
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.033675
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.062365
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.035269
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.064748
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.058646
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.118540
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.039569
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.081463
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.050964
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.035347
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.048129
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.013654
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.049576
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.019956
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.033429


Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.057822
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.097495
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.018915
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.041721
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.008618

Test set: Average loss: -14.1468, Accuracy: 9904/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.051740
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.005327
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.022074
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.023715
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.016285
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.016379
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.017268
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.016047
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.009732
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.014244
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.021623
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.060773
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.006663
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.015347
Tra

Train Epoch: 11 [16640/60000 (28%)]	Loss: 0.016415
Train Epoch: 11 [17920/60000 (30%)]	Loss: 0.006806
Train Epoch: 11 [19200/60000 (32%)]	Loss: 0.042001
Train Epoch: 11 [20480/60000 (34%)]	Loss: 0.009463
Train Epoch: 11 [21760/60000 (36%)]	Loss: 0.008479
Train Epoch: 11 [23040/60000 (38%)]	Loss: 0.038488
Train Epoch: 11 [24320/60000 (41%)]	Loss: 0.004094
Train Epoch: 11 [25600/60000 (43%)]	Loss: 0.022612
Train Epoch: 11 [26880/60000 (45%)]	Loss: 0.034368
Train Epoch: 11 [28160/60000 (47%)]	Loss: 0.005847
Train Epoch: 11 [29440/60000 (49%)]	Loss: 0.024630
Train Epoch: 11 [30720/60000 (51%)]	Loss: 0.010207
Train Epoch: 11 [32000/60000 (53%)]	Loss: 0.035756
Train Epoch: 11 [33280/60000 (55%)]	Loss: 0.025539
Train Epoch: 11 [34560/60000 (58%)]	Loss: 0.043341
Train Epoch: 11 [35840/60000 (60%)]	Loss: 0.018900
Train Epoch: 11 [37120/60000 (62%)]	Loss: 0.002289
Train Epoch: 11 [38400/60000 (64%)]	Loss: 0.003283
Train Epoch: 11 [39680/60000 (66%)]	Loss: 0.011887
Train Epoch: 11 [40960/60000 (6

Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.018388
Train Epoch: 14 [39680/60000 (66%)]	Loss: 0.004083
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.003056
Train Epoch: 14 [42240/60000 (70%)]	Loss: 0.009756
Train Epoch: 14 [43520/60000 (72%)]	Loss: 0.000960
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.033060
Train Epoch: 14 [46080/60000 (77%)]	Loss: 0.013747
Train Epoch: 14 [47360/60000 (79%)]	Loss: 0.001758
Train Epoch: 14 [48640/60000 (81%)]	Loss: 0.007716
Train Epoch: 14 [49920/60000 (83%)]	Loss: 0.025356
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.014542
Train Epoch: 14 [52480/60000 (87%)]	Loss: 0.006484
Train Epoch: 14 [53760/60000 (90%)]	Loss: 0.006068
Train Epoch: 14 [55040/60000 (92%)]	Loss: 0.029538
Train Epoch: 14 [56320/60000 (94%)]	Loss: 0.011548
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.029120
Train Epoch: 14 [58880/60000 (98%)]	Loss: 0.002874

Test set: Average loss: -16.1639, Accuracy: 9903/10000 (99%)

Train Epoch: 15 [0/60000 (0%)]	Loss: 0.004287
Train Epoch: 15 [1280/60

In [15]:
!rm -r mnist_data

'rm' is not recognized as an internal or external command,
operable program or batch file.
